**Otimização da resposta utilizando o método de evolução diferencial, combinado a rede neural de melhor setup inicial.**

In [0]:
# bibliotecas usadas
import os
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
%matplotlib inline

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [76]:
!git clone http://github.com/rodolfojbrandao/PhD
%cd PhD

Cloning into 'PhD'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 4283 (delta 10), reused 29 (delta 6), pack-reused 4247
Receiving objects: 100% (4283/4283), 53.85 MiB | 15.55 MiB/s, done.
Resolving deltas: 100% (365/365), done.
Checking out files: 100% (4054/4054), done.
/content/PhD/PhD/PhD/PhD


In [77]:
DAT=pd.read_csv("trabalho01redes2.csv")
print(DAT)
max_x1=DAT['pressure'].max()
min_x1=DAT['pressure'].min()
max_x2=DAT['temperature'].max()
min_x2=DAT['temperature'].min()
max_x3=DAT['co2'].max()
min_x3=DAT['co2'].min()
max_x4=DAT['is'].max()
min_x4=DAT['is'].min()
#guardar valore de a e b para cada variavel para facilitar retornar a normalização após a otimização
a1=(max_x1+min_x1)/2
b1=(max_x1-min_x1)/2
a2=(max_x2+min_x2)/2
b2=(max_x2-min_x2)/2
a3=(max_x3+min_x3)/2
b3=(max_x3-min_x3)/2
a4=(max_x4+min_x4)/2
b4=(max_x4-min_x4)/2

    pressure  temperature  co2        is
0        100           60  0.3  0.043365
1        300           50  0.2  0.068071
2        200           40  0.4  0.072746
3        200           60  0.2  0.061583
4        200           40  0.2  0.047607
5        300           60  0.3  0.100949
6        100           50  0.4  0.055266
7        100           50  0.2  0.026984
8        300           40  0.3  0.070051
9        200           60  0.4  0.133478
10       200           50  0.3  0.087583
11       100           40  0.3  0.021871
12       200           50  0.3  0.083618
13       200           50  0.3  0.071142
14       300           50  0.4  0.121988
15       300           50  0.3  0.056339
16       100           50  0.3  0.035870


In [0]:
# separa as colunas de interesse
Index=range(4)
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

In [0]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [0]:
# separa os dados em treino e teste
train=DAT.sample(frac=0.85,random_state=6) # poderia ser frac=0.5? sim ou nao?
test=DAT.drop(train.index)

In [0]:
# obtem as variaveis independentes (treino e teste)
x_train=train.iloc[:,[0,1,2]]
x_test=test.iloc[:,[0,1,2]]
X_OLD=DAT_OLD.iloc[:,[0,1,2]]

In [82]:
# obtem as variaveis dependentes (treino e teste)
Index=[3]
Y_OLD=DAT_OLD.iloc[:,Index]
y_train_Y=train.iloc[:,Index]
y_test__Y=test.iloc[:,Index]
print(Y_OLD.head())

         is
0  0.043365
1  0.068071
2  0.072746
3  0.061583
4  0.047607


In [0]:
# define a rede neural Y
clf_Y = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,hidden_layer_sizes=(6), 
                    random_state=3)

In [84]:
# treina a rede neural Y
clf_Y.fit(x_train, y_train_Y)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='tanh', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=6, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=3, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [0]:
# usa a rede neural treina em calculos para treino e teste Y
y_calc_train=clf_Y.predict(x_train)
x_test=x_train
y_calc_test=clf_Y.predict(x_test)


In [0]:
# transforma a saida da rede em dataframe 
y_calc_train=pd.DataFrame(y_calc_train)
y_calc_test=pd.DataFrame(y_calc_test)
col_names=list(Y_OLD)
y_calc_train.columns = col_names
y_calc_test.columns = col_names

In [0]:
# transforma dos dados calculados na forma original (nao normatizado)
y_calc_train=Original(y_calc_train,Y_OLD)
y_calc_test=Original(y_calc_test,Y_OLD)

In [0]:
def function(x):
  global clf_Y
  x=x.reshape((1, 3))
  df = pd.DataFrame(x)
  Y=clf_Y.predict(df)
  fun=Y*(-1) #maximizar
  return fun

In [0]:
bounds = [(-1, 1),(-1, 1),(-1, 1)]
OLD=Y_OLD
Condicoes = optimize.differential_evolution(function,bounds,
                                      init="latinhypercube", # obs: init="random"
                                      maxiter=100, popsize=50,recombination=0.5)
Condicoes=pd.DataFrame(Condicoes.x.reshape(1, 3))
#print(Condicoes)

In [90]:
otm_pressure=Condicoes[0]*b1+a1
otm_temperature=Condicoes[1]*b2+a2
otm_co2=Condicoes[2]*b3+a3
Y=clf_Y.predict(Condicoes)
otm_is=Y*b4+a4
print(otm_pressure)
print(otm_temperature)
print(otm_co2)
print(otm_is)

0    300.0
Name: 0, dtype: float64
0    60.0
Name: 1, dtype: float64
0    0.4
Name: 2, dtype: float64
[0.1803869]


pressure -- temperature -- co2 -- is 

expe:           200 -- 60 -- 0.4 -- 0.1334

ANN (artigo) :         283 -- 60 -- 0.4 -- 0.1385

**ANN (trabalho) :    300 -- 60 -- 0.4 -- 0.1804**